# 🛰️ StreetView Safety Analyzer

In [ ]:
# Install dependencies (uncomment if needed)
!pip install torch transformers pillow pandas google-streetview geopandas shapely folium ipyleaflet accelerate python-dotenv

import os
import glob
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, Point
from datetime import datetime
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image
import torch
from ipyleaflet import Map, DrawControl
from IPython.display import display
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")


In [5]:
import folium
from folium.plugins import Draw

m = folium.Map(location=[-6.2, 106.8], zoom_start=13)
Draw(export=True).add_to(m)
m  # <-- THIS must be the last line to render the map


In [25]:
import json
import osmnx as ox

# Load ROI GeoJSON
with open("data/ROI.geojson") as f:
    roi_geojson = json.load(f)

# Extract polygon from FeatureCollection
roi_polygon = shape(roi_geojson['features'][0]['geometry'])

# Use the polygon to get street network
G = ox.graph_from_polygon(roi_polygon, network_type='walk')


In [ ]:
# Install required packages before continuing
!pip install -q googlemaps osmnx networkx geopandas

In [ ]:


# Now we'll rewrite the full notebook in a clean and reusable structure

import os
import random
import time
import requests
import googlemaps
import osmnx as ox
import networkx as nx
from shapely.geometry import Point
from datetime import datetime
import pandas as pd

# --- CONFIGURATION ---

interval_meters = 50
num_coordinates = 20

# Use the polygon to get street network
G = ox.graph_from_polygon(roi_polygon, network_type='walk')
G = ox.distance.add_edge_lengths(G)
G = nx.Graph(G)

# Function to sample points from the street graph within the ROI
def get_street_points_from_graph(G, interval_meters=50, num_coordinates=10):
    sampled_points = []

    for u, v, data in G.edges(data=True):
        length = data.get('length', 0)
        num_samples = int(length // interval_meters)
        lat1, lon1 = G.nodes[u]['y'], G.nodes[u]['x']
        lat2, lon2 = G.nodes[v]['y'], G.nodes[v]['x']

        for i in range(1, num_samples + 1):
            frac = i / num_samples
            sample_lat = lat1 + frac * (lat2 - lat1)
            sample_lon = lon1 + frac * (lon2 - lon1)
            sampled_points.append((sample_lat, sample_lon))

    sampled_points = list(set(sampled_points))
    return random.sample(sampled_points, min(num_coordinates, len(sampled_points)))


# --- Step 2: Validate points via Google Maps API ---
def get_nearest_google_maps_point(coordinates, api_key):
    gmaps = googlemaps.Client(key=api_key)
    validated_points = set()

    for lat, lon in coordinates:
        try:
            result = gmaps.reverse_geocode((lat, lon))
            if result:
                validated_points.add((lat, lon))
            time.sleep(0.1)
        except Exception as e:
            print(f"Error fetching Google Maps data: {e}")

    return list(validated_points)

# Step 3: Download Street View Images.
def download_street_view_images(points, api_key, output_folder):
    gmaps = googlemaps.Client(key=api_key)
    os.makedirs(output_folder, exist_ok=True)
    metadata = []

    for i, (lat, lng) in enumerate(points):
        try:
            params = {
                "location": f"{lat},{lng}",
                "size": "640x640",
                "key": api_key
            }
            base_url = "https://maps.googleapis.com/maps/api/streetview"
            response = requests.get(base_url, params=params, stream=True)
            response.raise_for_status()

            image_path = os.path.join(output_folder, f"street_view_{i}.jpg")
            with open(image_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            metadata.append({"lat": lat, "lon": lng, "image": image_path})
            print(f"✅ Downloaded image {i+1}: {image_path}")
            time.sleep(0.4)

        except requests.exceptions.RequestException as e:
            print(f" Error downloading image {i+1}: {e}")
        except Exception as e:
            print(f" Unexpected error at image {i+1}: {e}")

    return metadata

# --- MAIN EXECUTION ---

# Create session folder
session_name = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
image_folder = os.path.join("images", session_name)
os.makedirs(image_folder, exist_ok=True)

# Step 1: Extract & Step 2: Validate coordinates
street_points = get_street_points_from_graph(G, interval_meters=50, num_coordinates=10)
print(f"Extracted {len(street_points)} street points.")

validated_points = get_nearest_google_maps_point(street_points, api_key)
print(f"Validated {len(validated_points)} points using Google Maps.")

# Step 3: Download Street View images
image_metadata = download_street_view_images(validated_points, api_key, image_folder)

# Step 4: Save metadata
os.makedirs("data", exist_ok=True)
#csv_path = os.path.join("data", f"streetview_metadata_{session_name}.csv")
#pd.DataFrame(image_metadata).to_csv(csv_path, index=False)



Extracted 10 street points.
Validated 10 points using Google Maps.
✅ Downloaded image 1: images\session_20250617_235704\street_view_0.jpg
✅ Downloaded image 2: images\session_20250617_235704\street_view_1.jpg
✅ Downloaded image 3: images\session_20250617_235704\street_view_2.jpg
✅ Downloaded image 4: images\session_20250617_235704\street_view_3.jpg
✅ Downloaded image 5: images\session_20250617_235704\street_view_4.jpg
✅ Downloaded image 6: images\session_20250617_235704\street_view_5.jpg
✅ Downloaded image 7: images\session_20250617_235704\street_view_6.jpg
✅ Downloaded image 8: images\session_20250617_235704\street_view_7.jpg
✅ Downloaded image 9: images\session_20250617_235704\street_view_8.jpg
✅ Downloaded image 10: images\session_20250617_235704\street_view_9.jpg


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForVision2Seq.from_pretrained(
    "llava-hf/llava-1.5-7b-hf", device_map="auto", torch_dtype=torch.float16
)

def describe_with_llava(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, text="Describe the street safety features in this image.", return_tensors="pt").to(device)
    generated = model.generate(**inputs, max_new_tokens=100)
    return processor.batch_decode(generated, skip_special_tokens=True)[0]


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

c:\Users\Quadro\anaconda3\envs\python_foundation\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Quadro\.cache\huggingface\hub\models--llava-hf--llava-1.5-7b-hf. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [52]:
latest_dir = sorted(glob.glob("images/session_*"))[-1]
image_paths = sorted(glob.glob(f"{latest_dir}/*.jpg"))

results = []
for path in image_paths:
    caption = describe_with_llava(path)
    results.append({"image": os.path.basename(path), "caption": caption})

df = pd.DataFrame(results)
df.to_csv(f"{latest_dir}/captions.csv", index=False)
df.head()


ValueError: Image features and image tokens do not match: tokens: 0, features 576

In [ ]:
def extract_safety_flags(text):
    return {
        "crosswalk": "crosswalk" in text.lower(),
        "light": "light" in text.lower() or "signal" in text.lower(),
        "pothole": "pothole" in text.lower() or "crack" in text.lower(),
        "daylight": "daylight" in text.lower() or "sun" in text.lower()
    }

flags_df = df["caption"].apply(extract_safety_flags).apply(pd.Series)
final_df = pd.concat([df, flags_df], axis=1)
final_df.to_csv(f"{latest_dir}/safety_flags.csv", index=False)
final_df


KeyError: 'caption'